In [12]:
import numpy as np
%load_ext autoreload
%autoreload 2

import torch
import sys

sys.path.append('..')

from algorithms.sequence_models.config import TrainConfig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is ", device)

In [13]:
from algorithms.sequence_models.decision_S4.dts4 import DecisionS4
from envs.umbrella_length import get_umbrella_length_env
from data.umbrella_length_dataset import UmbrellaLengthDataset
from comparison_experiment import AutomatedComparisonExperiment

chain_length = 5
env = get_umbrella_length_env(chain_length=chain_length)

max_eval_ep_len = chain_length + 2 # todo change for other envs
context_len = chain_length + 2  # todo change for other envs

config = TrainConfig(max_eval_ep_len=max_eval_ep_len, context_len=max_eval_ep_len)

n_trajectories = 10_000 # todo is this enough / too much?
experiment_name = f"experiment_umbrella_length_{chain_length}_dataset_size={n_trajectories}"
if UmbrellaLengthDataset.exists(experiment_name):
    traj_dataset = UmbrellaLengthDataset.load(experiment_name)
else:
    reward_scale = 1 # todo change for other envs
    traj_dataset = UmbrellaLengthDataset(env, n_trajectories=n_trajectories, reward_scale=reward_scale)
    traj_dataset.save(experiment_name)

comparison_experiment = AutomatedComparisonExperiment(
    env=env,
    traj_dataset=traj_dataset,
    config=config,
    device=device,
    rtgs_for_train_eval=np.linspace(-1.2, 1.2, 8, endpoint=True), # todo change for other envs
    rtgs_final_test=np.linspace(-1.2, 1.2, 15, endpoint=True), # todo change for other envs
    env_name=f"umbrella_length_{chain_length}", # todo change for other envs
    experiment_name="automated_comparison"
)

In [16]:
epochs = 10 # todo set to 200

In [17]:
comparison_experiment.run_and_report(comparison_experiment.dt_experiment, epochs=epochs)

In [18]:
comparison_experiment.run_and_report(comparison_experiment.s4_experiment, epochs=epochs)